<a href="https://colab.research.google.com/github/tumble-weed/LSTM-from-scratch/blob/master/LSTM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch

In [0]:
seq_len,batch_size,feat_len = 5,32,100
seq = np.random.random((seq_len,batch_size,feat_len))
seq = torch.tensor(seq).float()

In [0]:
class SimpleRNN(torch.nn.Module):
    '''
    Assumes the output is the state
    has just a single layer that deals with the state and input
    '''
    def __init__(self,input_len,output_len):
        super(SimpleRNN,self).__init__()
        
        state_len = output_len
        
        self.input_len,self.output_len,self.state_len = input_len,output_len,state_len
        self.fc_input = torch.nn.Linear(input_len,output_len)        
        self.fc_state = torch.nn.Linear(state_len,output_len,bias=False)
        
        pass
    
    def forward(self,x,state):        
        outs = []
        for at_t in x:

            out = self.fc_input(at_t) + self.fc_state(state)
            out = torch.nn.functional.relu(out)
            state = out
            outs.append(out)
        outs = torch.stack(outs,0)
        return outs,state

        

In [0]:
input_len,output_len = feat_len,9
simple_rnn = SimpleRNN(input_len,output_len)
hidden = torch.zeros((1,batch_size,output_len))
out,state =simple_rnn(seq,hidden)

In [0]:
out.shape,state.shape

(torch.Size([5, 1, 32, 9]), torch.Size([1, 32, 9]))

In [0]:
class RNN(torch.nn.Module):
    def __init__(self,):
        pass
    def forward(self,):
        pass


In [0]:
class ProtoLSTM_OutputIsState(torch.nn.Module):
    def __init__(self,input_len,output_len):
        
        torch.nn.Module.__init__(self)
        
        hidden_len = output_len
        
        self.hidden_len = hidden_len
        self.output_len = output_len
        self.input_len = input_len
        #---------------------------------------------------------
        # Gates
        #---------------------------------------------------------
        self.remember_state = torch.nn.Linear(hidden_len,hidden_len)
        self.remember_input = torch.nn.Linear(input_len,hidden_len,bias=False)
        
        self.write_state = torch.nn.Linear(hidden_len,hidden_len)
        self.write_input = torch.nn.Linear(input_len,hidden_len,bias=False)
        
        self.read_state = torch.nn.Linear(hidden_len,hidden_len)
        self.read_input = torch.nn.Linear(input_len,hidden_len,bias=False)
        #---------------------------------------------------------
        # fc
        #---------------------------------------------------------
        self.fc_input = torch.nn.Linear(input_len,output_len)
        self.fc_state = torch.nn.Linear(hidden_len,output_len,bias = False)
    def forward(self,x,state):
        outs = []
        for at_t in x:
            read = torch.nn.functional.sigmoid(self.read_state(state)+self.read_input(at_t))
            write = torch.nn.functional.sigmoid(self.write_state(state)+self.write_input(at_t))
            remember = torch.nn.functional.sigmoid(self.remember_state(state)+self.remember_input(at_t))
            
            pre_state = self.fc_state(state*read) + self.fc_input(at_t)
            pre_state = torch.nn.functional.tanh(pre_state)
            state = write*pre_state + remember*state
            out = state
            outs.append(out)
        outs = torch.stack(outs,0)
        return outs,state
    

In [0]:

input_len,output_len = feat_len,9
lstm1 = ProtoLSTM_OutputIsState(input_len,output_len)
hidden = torch.zeros((1,batch_size,output_len))
out,state =lstm1(seq,hidden)
print(out.shape,state.shape)

torch.Size([5, 1, 32, 9]) torch.Size([1, 32, 9])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [0]:
class GRU_OutputIsState(torch.nn.Module):
    def __init__(self,input_len,output_len):
        
        torch.nn.Module.__init__(self)
        
        hidden_len = output_len
        
        self.hidden_len = hidden_len
        self.output_len = output_len
        self.input_len = input_len
        #---------------------------------------------------------
        # Gates
        #---------------------------------------------------------
        self.write_gate = torch.nn.Linear(hidden_len,hidden_len)
        self.read_gate = torch.nn.Linear(hidden_len,hidden_len)
        #---------------------------------------------------------
        # fc
        #---------------------------------------------------------
        self.fc_input = torch.nn.Linear(input_len,output_len)
        self.fc_state = torch.nn.Linear(hidden_len,output_len,bias = False)
    
    def forward(self,x,state):
        outs = []
        for at_t in x:
            read = torch.nn.functional.sigmoid(self.read_gate(state))
            write = torch.nn.functional.sigmoid(self.write_gate(state))
            
            pre_state = self.fc_state(state*read) + self.fc_input(at_t)
            state = write*pre_state + (1-write)*state
            out = state
            outs.append(out)
        outs = torch.stack(outs,0)
        return outs,state